In [ ]:
import pygame
import sys
import random
import time

WIDTH, HEIGHT = 900, 720
FPS = 60

BOARD_LEFT = 50
BOARD_TOP = 50
BOARD_SIZE = 600
CELL = BOARD_SIZE // 10

DICE_SIZE = 120
INFO_X = BOARD_LEFT + BOARD_SIZE + 20
INFO_Y = BOARD_TOP

PLAYER_RADIUS = 12

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BOARD_COLORS = [
    (244, 67, 54), (33, 150, 243), (255, 235, 59),
    (156, 39, 176), (76, 175, 80), (255, 152, 0)
]
PLAYER_COLORS = [(220, 20, 60), (20, 120, 200)]

DEFAULT_LADDERS = {3:22, 8:30, 28:84}
DEFAULT_SNAKES  = {98:41, 64:36, 56:19}

def get_square_center(n):
    if n < 1: n = 1
    if n > 100: n = 100
    idx = n - 1
    row = idx // 10
    col = idx % 10
    cx = BOARD_LEFT + (col * CELL)
    cy = BOARD_TOP + (9 - row) * CELL
    if row % 2 == 1:
        cx = BOARD_LEFT + ((9 - col) * CELL)
    return (cx + CELL // 2, cy + CELL // 2)

def draw_text_center(surface, txt, font, color, rect):
    text = font.render(txt, True, color)
    tr = text.get_rect(center=rect.center)
    surface.blit(text, tr)

def prompt_pairs(name, kind):
    print(f"\nEnter 3 {name} pairs. Format example: 3,22 8,30 28,84")
    print("Press Enter for defaults.")
    raw = input(f"{name} pairs: ").strip()
    if not raw:
        return None
    try:
        parts = raw.split()
        if len(parts) != 3:
            print("Enter exactly 3 pairs. Using defaults.")
            return None
        d = {}
        for p in parts:
            s,e = p.split(",")
            s = int(s); e = int(e)
            if s < 2 or s > 99 or e < 2 or e > 99:
                print("Values 2–99 only. Using defaults.")
                return None
            if kind == 'ladder' and not (s < e):
                print("Ladders must be s<e. Using defaults.")
                return None
            if kind == 'snake' and not (s > e):
                print("Snakes must be s>e. Using defaults.")
                return None
            d[s] = e
        if len(d) != 3:
            print("Duplicate start. Using defaults.")
            return None
        return d
    except:
        print("Invalid input. Using defaults.")
        return None

def draw_board(surface, font_small):
    for n in range(1, 101):
        cx, cy = get_square_center(n)
        rect = pygame.Rect(cx - CELL // 2, cy - CELL // 2, CELL, CELL)
        color = BOARD_COLORS[((n-1)//10 + (n-1)%10) % len(BOARD_COLORS)]
        pygame.draw.rect(surface, color, rect)
        pygame.draw.rect(surface, BLACK, rect, 1)
        num_surf = font_small.render(str(n), True, BLACK)
        surface.blit(num_surf, (rect.x + 4, rect.y + 2))

def draw_snakes_ladders(surface, snakes, ladders):
    for start,end in ladders.items():
        sx, sy = get_square_center(start)
        ex, ey = get_square_center(end)
        pygame.draw.line(surface, (139,69,19), (sx-10,sy-10), (ex-10,ey-10), 6)
        pygame.draw.line(surface, (139,69,19), (sx+10,sy+10), (ex+10,ey+10), 6)
        steps = 8
        dx = ex - sx
        dy = ey - sy
        for i in range(1, steps):
            t = i/steps
            rx = sx + dx * t
            ry = sy + dy * t
            pygame.draw.line(surface, (222,184,135), (rx-12, ry-12), (rx+12, ry+12), 4)
    for start,end in snakes.items():
        sx, sy = get_square_center(start)
        ex, ey = get_square_center(end)
        points = []
        midx, midy = (sx+ex)/2, (sy+ey)/2
        offset = 40
        points.append((sx, sy))
        points.append((sx + offset, sy - offset))
        points.append((midx - offset, midy + offset))
        points.append((ex - offset, ey - offset))
        points.append((ex, ey))
        pygame.draw.lines(surface, (0, 120, 0), False, points, 10)
        pygame.draw.circle(surface, (0,150,0), (int(sx), int(sy)), 14)
        pygame.draw.circle(surface, BLACK, (int(sx+6), int(sy-6)), 3)

def draw_players(surface, positions):
    offsets = [(-10,-10), (10,10)]
    for i,pos in enumerate(positions):
        cx, cy = get_square_center(pos)
        ox, oy = offsets[i]
        pygame.draw.circle(surface, PLAYER_COLORS[i], (cx+ox, cy+oy), PLAYER_RADIUS)
        pygame.draw.circle(surface, BLACK, (cx+ox, cy+oy), PLAYER_RADIUS, 2)

def main(snakes, ladders):
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Snakes & Ladders")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont(None, 28)
    font_big = pygame.font.SysFont(None, 36)
    font_small = pygame.font.SysFont(None, 20)

    positions = [1, 1]
    turn = 0
    last_roll = None
    winner = None

    roll_button = pygame.Rect(INFO_X + 10, INFO_Y + 200, DICE_SIZE, 40)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                mx,my = event.pos
                if roll_button.collidepoint((mx,my)) and winner is None:
                    for i in range(12):
                        anim_roll = random.randint(1,6)
                        screen.fill(WHITE)
                        draw_board(screen, font_small)
                        draw_snakes_ladders(screen, snakes, ladders)
                        draw_players(screen, positions)
                        screen.blit(font_big.render(str(anim_roll), True, BLACK), (INFO_X + 40, INFO_Y + 100))
                        pygame.draw.rect(screen, (180,180,180), roll_button)
                        screen.blit(font.render("ROLL DICE", True, BLACK), (roll_button.x+10, roll_button.y+8))
                        pygame.display.flip()
                        pygame.time.delay(80)
                    last_roll = random.randint(1,6)
                    target = positions[turn] + last_roll
                    if target > 100:
                        target = positions[turn]
                    while positions[turn] != target:
                        positions[turn] += 1
                        screen.fill(WHITE)
                        draw_board(screen, font_small)
                        draw_snakes_ladders(screen, snakes, ladders)
                        draw_players(screen, positions)
                        pygame.display.flip()
                        pygame.time.delay(120)
                    if positions[turn] in ladders:
                        dest = ladders[positions[turn]]
                        sx, sy = get_square_center(positions[turn])
                        ex, ey = get_square_center(dest)
                        for t in range(1, 12):
                            tt = t / 12
                            cx = int(sx + (ex - sx) * tt)
                            cy = int(sy + (ey - sy) * tt - 30*(1 - (2*tt-1)**2))
                            screen.fill(WHITE)
                            draw_board(screen, font_small)
                            draw_snakes_ladders(screen, snakes, ladders)
                            for i,p in enumerate(positions):
                                if i == turn:
                                    pygame.draw.circle(screen, PLAYER_COLORS[i], (cx, cy), PLAYER_RADIUS)
                                    pygame.draw.circle(screen, BLACK, (cx, cy), PLAYER_RADIUS, 2)
                                else:
                                    pcx, pcy = get_square_center(p)
                                    pygame.draw.circle(screen, PLAYER_COLORS[i], (pcx, pcy), PLAYER_RADIUS)
                                    pygame.draw.circle(screen, BLACK, (pcx, pcy), PLAYER_RADIUS, 2)
                            pygame.display.flip()
                            pygame.time.delay(40)
                        positions[turn] = dest
                    elif positions[turn] in snakes:
                        dest = snakes[positions[turn]]
                        sx, sy = get_square_center(positions[turn])
                        ex, ey = get_square_center(dest)
                        for t in range(1, 12):
                            tt = t / 12
                            cx = int(sx + (ex - sx) * tt)
                            cy = int(sy + (ey - sy) * tt + 30*(1 - (2*tt-1)**2))
                            screen.fill(WHITE)
                            draw_board(screen, font_small)
                            draw_snakes_ladders(screen, snakes, ladders)
                            for i,p in enumerate(positions):
                                if i == turn:
                                    pygame.draw.circle(screen, PLAYER_COLORS[i], (cx, cy), PLAYER_RADIUS)
                                    pygame.draw.circle(screen, BLACK, (cx, cy), PLAYER_RADIUS, 2)
                                else:
                                    pcx, pcy = get_square_center(p)
                                    pygame.draw.circle(screen, PLAYER_COLORS[i], (pcx, pcy), PLAYER_RADIUS)
                                    pygame.draw.circle(screen, BLACK, (pcx, pcy), PLAYER_RADIUS, 2)
                            pygame.display.flip()
                            pygame.time.delay(40)
                        positions[turn] = dest
                    if positions[turn] == 100:
                        winner = turn
                    else:
                        turn = 1 - turn

        screen.fill(WHITE)
        draw_board(screen, font_small)
        draw_snakes_ladders(screen, snakes, ladders)
        draw_players(screen, positions)

        screen.blit(font_big.render("Snakes & Ladders", True, BLACK), (INFO_X, INFO_Y))
        screen.blit(font.render(f"Turn: Player {turn+1}", True, BLACK), (INFO_X, INFO_Y+50))
        screen.blit(font.render(f"Last roll: {last_roll if last_roll else '-'}", True, BLACK), (INFO_X, INFO_Y+90))

        screen.blit(font.render("Ladders:", True, BLACK), (INFO_X, INFO_Y+140))
        offy = INFO_Y + 170
        for k,v in ladders.items():
            screen.blit(font.render(f"{k} -> {v}", True, BLACK), (INFO_X, offy))
            offy += 22
        screen.blit(font.render("Snakes:", True, BLACK), (INFO_X, offy+6))
        offy += 28
        for k,v in snakes.items():
            screen.blit(font.render(f"{k} -> {v}", True, BLACK), (INFO_X, offy))
            offy += 22

        pygame.draw.rect(screen, (200,200,200), roll_button)
        screen.blit(font.render("ROLL DICE", True, BLACK), (roll_button.x+10, roll_button.y+8))

        if winner is not None:
            screen.blit(font_big.render(f"Player {winner+1} WINS!!!", True, (255,0,0)), (INFO_X, INFO_Y + 420))

        pygame.display.flip()
        clock.tick(FPS)

if __name__ == "__main__":
    print("Snakes & Ladders (Pygame)")
    ladders = prompt_pairs("Ladder", "ladder")
    snakes = prompt_pairs("Snake", "snake")
    if ladders is None:
        ladders = DEFAULT_LADDERS.copy()
    if snakes is None:
        snakes = DEFAULT_SNAKES.copy()

    conflict = False
    for s in ladders.keys():
        if s in snakes:
            conflict = True
    if conflict:
        print("Conflict detected. Using defaults.")
        ladders = DEFAULT_LADDERS.copy()
        snakes = DEFAULT_SNAKES.copy()

    valid = True
    for s,e in ladders.items():
        if not (2 <= s < e <= 99): valid = False
    for s,e in snakes.items():
        if not (2 <= e < s <= 99): valid = False
    if not valid:
        print("Invalid values. Using defaults.")
        ladders = DEFAULT_LADDERS.copy()
        snakes = DEFAULT_SNAKES.copy()

    main(snakes, ladders)


pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Snakes & Ladders (Pygame)

Enter 3 Ladder pairs. Format example: 3,22 8,30 28,84
Press Enter for defaults.
